In [10]:
!pip install ultralytics

In [11]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO 

In [12]:
model = YOLO("yolov8n.pt")

In [13]:
video_path = "cr (1).mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

In [14]:
ACCIDENT_THRESHOLD = 50

In [15]:
def detect_accident(prev_frame, current_frame):
    """Detects if an accident has occurred based on frame differences."""
    diff = cv2.absdiff(prev_frame, current_frame)
    gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    retval, thresh = cv2.threshold(gray_diff, 30, 255, cv2.THRESH_BINARY)  
    accident_score = np.sum(thresh) / 255 
    cv2.imshow("Frame Difference", gray_diff)
    return accident_score > ACCIDENT_THRESHOLD 

In [16]:
prev_frame = None 

In [17]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break
    print(f"Processing frame {cap.get(cv2.CAP_PROP_POS_FRAMES)}") 
    results = model(frame)
    detections = results[0].boxes.data.cpu().numpy()  
    
    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection  
        label = model.names[int(cls)]
        
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    if prev_frame is not None:
        if frame.shape != prev_frame.shape:  
            prev_frame = cv2.resize(prev_frame, (frame.shape[1], frame.shape[0]))

        if detect_accident(prev_frame, frame):
            print("⚠️ Accident Detected! Sending Alert...")
            cv2.putText(frame, "ACCIDENT DETECTED!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    prev_frame = frame.copy()  

    cv2.imshow("Smart Traffic & Accident Prevention", frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Processing frame 1.0

0: 384x640 3 cars, 127.6ms
Speed: 8.6ms preprocess, 127.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2.0

0: 384x640 3 cars, 143.9ms
Speed: 6.5ms preprocess, 143.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
⚠️ Accident Detected! Sending Alert...
Processing frame 3.0

0: 384x640 3 cars, 148.4ms
Speed: 3.6ms preprocess, 148.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
⚠️ Accident Detected! Sending Alert...
Processing frame 4.0

0: 384x640 3 cars, 124.8ms
Speed: 4.0ms preprocess, 124.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
⚠️ Accident Detected! Sending Alert...
Processing frame 5.0

0: 384x640 5 cars, 142.8ms
Speed: 3.9ms preprocess, 142.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
⚠️ Accident Detected! Sending Alert...
Processing frame 6.0

0: 384x640 5 cars, 129.4ms
Speed: 3.4ms preprocess, 129.4ms inference, 1.8ms postproces

In [9]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
from ultralytics import YOLO

# Load the trained YOLOv8 model (update path if needed)
model = YOLO('runs/detect/train/weights/best.pt')

# Evaluate on validation set
metrics = model.val()

# Print evaluation metrics
print("Precision:", metrics.box.precision)
print("Recall:", metrics.box.recall)
print("mAP50:", metrics.box.map50)
print("mAP50-95:", metrics.box.map)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os

results_dir = 'runs/detect/train'
results_csv = os.path.join(results_dir, 'results.csv')

# Plot training loss if CSV exists
if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)
    df[['      epoch', 'box_loss', 'cls_loss', 'dfl_loss']].plot(figsize=(10, 6), title='Loss During Training')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.show()
else:
    print("results.csv not found. Please ensure training results exist.")